In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pickle

import warnings
warnings.filterwarnings("ignore")

In [15]:
train_file = pd.read_csv('Fitbee-Campaigns-Jan-1-2021-Jun-30-2021 (1).csv')

In [16]:
train_file.head()

,Reporting Starts,Reporting Ends,Campaign Name,Campaign Product Name,Region,Campaign Delivery,Ad Set Budget,Ad Set Budget Type,Results,Result Indicator,Reach,Impressions,Cost per Results,Amount Spent (INR),Ends,Page Likes
0,01-01-2021,30-06-2021,Instagram Lookalike DM 5.11.2020,Instagram Lookalike,Delhi,inactive,1000,Daily,1.0,actions:onsite_conversion.messaging_conversati...,0,0,NaN,0.000000,Ongoing,NaN
1,01-01-2021,30-06-2021,Instagram Lookalike DM 5.11.2020,Instagram Lookalike,Karnataka,inactive,1000,Daily,NaN,NaN,0,0,NaN,0.000000,Ongoing,NaN
2,01-01-2021,30-06-2021,4 Week Plan 04.12.2020,The 4 week Plan,Andaman and Nicobar Islands,inactive,800,Daily,NaN,NaN,128,129,NaN,1.919988,Ongoing,NaN
3,01-01-2021,30-06-2021,4 Week Plan 04.12.2020,The 4 week Plan,Andhra Pradesh,inactive,800,Daily,17.0,actions:onsite_conversion.messaging_conversati...,8896,11150,13.904620,236.378547,Ongoing,NaN
4,01-01-2021,30-06-2021,4 Week Plan 04.12.2020,The 4 week Plan,Assam,inactive,800,Daily,1.0,actions:onsite_conversion.messaging_conversati...,1984,2956,45.339721,45.339721,Ongoing,NaN


In [17]:
train_file= train_file.fillna(method='ffill')

In [18]:
train_file.shape

(2214, 16)

In [19]:
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2214 entries, 0 to 2213
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Reporting Starts       2214 non-null   object 
 1   Reporting Ends         2214 non-null   object 
 2   Campaign Name          2214 non-null   object 
 3   Campaign Product Name  2214 non-null   object 
 4   Region                 2214 non-null   object 
 5   Campaign Delivery      2214 non-null   object 
 6   Ad Set Budget          2214 non-null   int64  
 7   Ad Set Budget Type     2214 non-null   object 
 8   Results                2214 non-null   float64
 9   Result Indicator       2214 non-null   object 
 10  Reach                  2214 non-null   int64  
 11  Impressions            2214 non-null   int64  
 12  Cost per Results       2211 non-null   float64
 13  Amount Spent (INR)     2214 non-null   float64
 14  Ends                   2214 non-null   object 
 15  Page

In [21]:
train_x = train_file[[ 'Campaign Product Name', 'Region', 'Amount Spent (INR)']]
train_y = train_file['Results']

In [22]:
train_x.head()

,Campaign Product Name,Region,Amount Spent (INR)
0,Instagram Lookalike,Delhi,0.000000
1,Instagram Lookalike,Karnataka,0.000000
2,The 4 week Plan,Andaman and Nicobar Islands,1.919988
3,The 4 week Plan,Andhra Pradesh,236.378547
4,The 4 week Plan,Assam,45.339721


In [26]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

numeric_features = ['Amount Spent (INR)']
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])    

categorical_features = ['Campaign Product Name', 'Region']
categorical_transformer = OneHotEncoder(drop='first', sparse=False)

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                               ('cat', categorical_transformer, categorical_features)])

In [27]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(random_state=100))])

In [28]:
clf.fit(train_x, train_y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Amount Spent (INR)']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['Campaign Product Name',
                                                   'Region'])])),
                ('classifier', LogisticRegression(random_state=100))])

In [29]:
pickle.dump(clf, open('model.pkl', 'wb'))